In [20]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import idx2numpy
import warnings
import random
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score, homogeneity_score, silhouette_score

In [21]:
# read dataset
x_train = pd.DataFrame(idx2numpy.convert_from_file("fashion_data/train-images-idx3-ubyte").reshape(60000, 784))
y_train = pd.DataFrame(idx2numpy.convert_from_file("fashion_data/train-labels-idx1-ubyte"), columns=['label'])
x_test = pd.DataFrame(idx2numpy.convert_from_file("fashion_data/t10k-images-idx3-ubyte").reshape(10000, 784))
y_test = pd.DataFrame(idx2numpy.convert_from_file("fashion_data/t10k-labels-idx1-ubyte"), columns=['label'])

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000, 1)
(10000, 784)
(10000, 1)


In [22]:
# concate data
x_train1 = pd.DataFrame(x_train)
y_train1= pd.DataFrame(y_train, columns=['label'])
x_test1 = pd.DataFrame(x_test)
y_test1= pd.DataFrame(y_test, columns=['label'])

df1 = pd.concat([x_train1, y_train1], axis=1)
df2 = pd.concat([x_test1, y_test1], axis=1)
df = pd.concat([df1, df2], axis=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
1,0,0,0,0,0,1,0,0,0,0,...,114,130,76,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,22,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## split the data

In [23]:
# extract feature and target from dataframe
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# scale the data
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))


# split data into train validation and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=1)

In [24]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(len(y_train.unique()))
temp = X_train[8000:15000]

(49000, 784)
(7000, 784)
(14000, 784)
10


## Heirarchical clustering 

In [25]:
# heirarchical clustering with 10 cluster
model = AgglomerativeClustering(n_clusters=10)
x_train_pred = model.fit(temp)
print("train set silhouette_score for hierarchical clustering ", silhouette_score(temp, x_train_pred.labels_))

train set silhouette_score for hierarchical clustering  0.10489096142249621


In [14]:
X_val_pred = model.fit(X_val)
print("validation set silhouette_score for hierarchical clustering ", silhouette_score(X_val, X_val_pred.labels_))

validation set silhouette_score for hierarchical clustering  0.09784989819696001


In [15]:
X_test_pred = model.fit(X_test)
print("test set silhouette_score for hierarchical clust
      ering ", silhouette_score(X_test, X_test_pred.labels_))

test set silhouette_score for hierarchical clustering  0.08683623613604623


## DBSCAN clustering

In [16]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=19, min_samples=5) 
clustering.fit(temp)
print("train set silhouette_score is", silhouette_score(temp, clustering.labels_))

train set silhouette_score is 0.17761589822182308


In [26]:
clustering = DBSCAN(eps=19, min_samples=5) 
clustering.fit(X_val)
print("validation set silhouette_score is", silhouette_score(X_val, clustering.labels_))

validation set silhouette_score is 0.06527834433629418


In [28]:
clustering = DBSCAN(eps=19, min_samples=5) 
clustering.fit(X_test)
print("test set silhouette_score is", silhouette_score(X_test, clustering.labels_))

test set silhouette_score is -0.013645788311733515


In [32]:
# print(set(clustering.labels_))
# noise = list(clustering.labels_).count(-1)
# noise

## ANALYSIS

Hierarchical clustering perfroms better than DBSCAN clustering on given datasets.
Becauses as we can see from above that in DBSCAN algorithm there is negative score means clusters are overlapping.
Wehreas in Hierarchical clustering scores are positive so no overlap of clusters.